In [1]:
# pip install openai

In [8]:
import json
import requests
import openai
import os
import time

In [6]:
api_key = 'YOUR_API_KEY'
file_path = '../Data/kaggle_data_first_page_2.json'
# file_path_2 = '../Data/kaggle_data_first_page_2.json'

In [17]:
with open(file_path, 'r') as file:
        comp_data = json.load(file)

comp_data[0]

{'name': 'CPROD1: Consumer PRODucts contest #1',
 'url': 'https://www.kaggle.com/competitions/cprod1',
 'overview_text': 'Overview text not found',
 'description_text': "A significant proportion of web usage relates to discussions, research, and purchase of consumer products. Currently, hundreds of thousands of blogs, forums, product review sites, and e-commerce merchants currently exist, in part, to service consumer's need to access product related information and demand to share experiences with products. The goal of this competition is to determine the state-of-the-art methods to automatically recognize product mentions in such textual content and to also disambiguate which product(s) in product catalogs are being referenced. Specifically, the task is to automatically identify all mentions of consumer products in a largely user generated collection of web-content, and to correctly identify the product(s) that each product mention refers to from a large catalog of products. The datas

In [21]:
count = sum(1 for d in comp_data if d.get('overview_text') == 'Overview text not found')
print(f"Number of entries with 'overview_text' as 'Overview text not found': {count}")

Number of entries with 'overview_text' as 'Overview text not found': 280


- Examine the data we scraped

In [19]:
with open(file_path, 'r') as file:
    comp_data = json.load(file)

name = comp_data[0]['name']
overview = comp_data[0]['overview_text']
description = comp_data[0]['description_text']
dataset = comp_data[0]['dataset_text']

In [20]:
len(comp_data)

300

In [28]:
# def query_chatgpt(name, overview, description, dataset, api_key):
#     # prompt = f"Analyze the information of the following data competition {name} with the overview of the competition being: '{overview}', the competition description of the competition being: '{description}', and the dataset description being: '{dataset}'.\n" \
#     #             "Provide a JSON structured output with fields for 'category' (which field or industry this is about) and 'fairness_bias_mentioned' (yes or no, depending on whether fairness and bias of model and dataset is explicitly mentioned in the descriptions or overview), \n" \
#     #             "please be carefully advised that Fairness in data science refers to the equitable and just treatment of individuals by algorithms and models. It focuses on ensuring that decisions made by automated systems do not favor one group of individuals over another on the basis of sensitive characteristics such as race, gender, age, or other protected attributes.\n"\
#     #             "Bias in data science refers to systemic deviation in the data or model that leads to less accurate, unfair, or unethical outcomes. Please consider these definitions to give the answer if this competition explicitly cares about fairness and bias\n" \
#     #             "and the field 'how' on how fairness and bias is mentioned (summarize within 50 words, n/a if not mentioned), the field 'toy' on whether this is a toy competition, meaning it has no real impact but only for fun (yes or no), also the field 'red_team', indicating if the competition is a red teaming competition (yes or no).\n" \
#     #             "Use double quotes for keys and string values! Do not include any other text except JSON itself!!! Do not include in front of JSON content itself!!! DO NOT include the string 'json' anywhere in the answer!!! \n" \
#     #             "DO NOT include the string 'json' anywhere in the answer!!! I DO NOT WANNA SEE 'json' anywhere in this answer! 'json' is a forbidden word in the answer!!!"
#     prompt = f"Analyze the information of the following data competition {name}, the competition description of the competition is: '{description}', and the dataset description is: '{dataset}'.\n" \
#                 "Provide a JSON structured output with fields for 'category' (which field or industry this is about) and 'fairness_bias_mentioned' (yes or no, depending on whether fairness and bias of model and dataset is explicitly mentioned in the descriptions or overview), \n" \
#                 "please be advised that Fairness in data science refers to the equitable and just treatment of individuals by algorithms and models. It focuses on ensuring that decisions made by automated systems do not favor one group of individuals over another on the basis of sensitive characteristics such as race, gender, age, or other protected attributes.\n"\
#                 "Bias in data science refers to systemic deviation in the data or model that leads to less accurate, unfair, or unethical outcomes. Please consider these definitions to give the answer if this competition explicitly cares about fairness and bias\n" \
#                 "and the field 'how' on how fairness and bias is mentioned (summarize within 50 words, n/a if not mentioned), the field 'toy' on whether this is a toy competition, meaning it has no real impact but only for fun or practices (yes or no), also the field 'red_team', indicating if the competition is a red teaming competition (yes or no).\n" \
#                 "Analyze '{dataset}' following 'category' and 'fairness_bias_mentioned', create another field 'data_privacy' (yes or no, depending on whether you think the data used in the competition has data privacy issues, such as having sensitive personal information, unauthorized use of data, or data breaches). \n" \
#                 "Following the yes or no in 'data_privacy', summarize with 50 words what potential data privacy issues the competition has, and if so, how is this dealt with, are there flaws in the approach. \n" \
#                 "Use double quotes for keys and string values! Do not include any other text except JSON itself! DO NOT include the string 'json' anywhere in the answer! \n" \
#                 "DO NOT include the string 'json' anywhere in the answer! I DO NOT WANNA SEE 'json' anywhere in this answer! Again, 'json' is a forbidden word in the answer!!!"
#     response = openai.ChatCompletion.create(
#         model="gpt-4o",
#         messages=[
#             {"role": "system", "content": "You are an assistant trained to analyze comments and provide structured outputs."},
#             {"role": "user", "content": prompt}
#         ],
#         api_key=api_key
#     )

#     generated_text = response['choices'][0]['message']['content'].strip()
#     print(generated_text)
#     return json.loads(generated_text)

In [38]:
def query_chatgpt(name, overview, description, dataset, api_key):
    client = openai.OpenAI(api_key=api_key)

    prompt = f"""
        Analyze the following data competition named {name}. The competition description is: '{description}', and the dataset description is: '{dataset}'.

        Provide a structured output in JSON style with the following fields:

        - "category": the field or industry this competition belongs to.
        - "fairness_bias_mentioned": yes or no, depending on whether fairness and bias of the model and dataset are explicitly mentioned in the descriptions or overview.
        - "how_fairness": a summary (within 50 words) describing how fairness and bias are mentioned, or "n/a" if not mentioned.
        - "data_privacy": yes or no, depending on whether you think the data used in the competition has data privacy issues (e.g., sensitive personal information, unauthorized data use, data breaches).
        - "how_data_privacy": summarize (within 50 words) what potential data privacy issues exist, or "n/a" if there are none. If privacy issues are present, explain how they are addressed and whether there are flaws in the approach.
        - "toy": yes or no, depending on whether this is a toy competition (meant for fun or practice only, no real-world impact).
        - "red_team": yes or no, indicating if the competition is a red teaming competition.
        - "transparency_mentioned": yes or no, depending on whether model transparency and explainability are explicitly mentioned as evaluation criteria.
        - "how_transparency": summarize (within 50 words) how transparency or explainability is discussed, or "n/a" if not mentioned. If the competition only focuses on a single best performance metric without considering model transparency, tell me what the metric is after "n/a".
        
        Please use double quotes for all keys and string values! Do not include any other text besides the structured output! DO NOT include the string 'json' anywhere in the answer! I DO NOT WANT TO SEE 'json' anywhere in the answer! Again, 'json' is a forbidden word in the answer!!!
        """
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an assistant trained to analyze comments and provide structured outputs."},
            {"role": "user", "content": prompt}
        ]
    )

    generated_text = response.choices[0].message.content.strip()
    print(generated_text)
    return json.loads(generated_text)


In [41]:
result = query_chatgpt(name, overview, description, dataset, api_key)

{
  "category": "Consumer Products and E-Commerce",
  "fairness_bias_mentioned": "no",
  "how_fairness": "n/a",
  "data_privacy": "no",
  "how_data_privacy": "n/a",
  "toy": "no",
  "red_team": "no",
  "transparency_mentioned": "no",
  "how_transparency": "n/a: The competition focuses on identifying and linking product mentions without mention of transparency or explainability criteria."
}


In [42]:
# def process_comments(file_path, api_key):
#     with open(file_path, 'r') as file:
#         comments_data = json.load(file)
    
#     results = []
#     positives = []

#     for item in comments_data:
#         dict_result = query_chatgpt(item['name'], item['overview_text'], item['description_text'], item['dataset_text'], api_key)
#         dict_result['name'] = item['name']
#         dict_result['url'] = item['url']
#         results.append(dict_result)
#         if dict_result['fairness_bias_mentioned'] == 'yes':
#             positives.append(dict_result)

#     with open('../Data/processed_comments_1.json', 'w') as outfile:
#         json.dump(results, outfile, indent=4)
#     return results, positives

# def process_comments(file_path, api_key):
#     with open(file_path, 'r') as file:
#         comments_data = json.load(file)
    
#     results = []
#     positives = []

#     for item in comments_data[:100]:
#         try:
#             dict_result = query_chatgpt(item['name'], item['overview_text'], item['description_text'], item['dataset_text'], api_key)
#             dict_result['name'] = item['name']
#             dict_result['url'] = item['url']
#             results.append(dict_result)
#             if dict_result.get('fairness_bias_mentioned') == 'yes':
#                 positives.append(dict_result)
#         except Exception as e:
#             # Handle the error by logging or any other method deemed appropriate
#             print(f"Failed to process item {item['name']} due to an error: {str(e)}")
#             # Optionally, you could append partial results or error information to results list here

#     with open('../Data/processed_comments_1.json', 'w') as outfile:
#         json.dump(results, outfile, indent=4)
    
#     return results, positives

# if __name__ == "__main__":
#     results, positives = process_comments(file_path, api_key)

def process_comments(file_path, api_key):
    with open(file_path, 'r') as file:
        comments_data = json.load(file)
    
    results = []
    positives = []

    for item in comments_data:
        retry_attempts = 3  # Set the number of retry attempts
        for attempt in range(retry_attempts):
            try:
                dict_result = query_chatgpt(item['name'], item['overview_text'], item['description_text'], item['dataset_text'], api_key)
                dict_result['name'] = item['name']
                dict_result['url'] = item['url']
                results.append(dict_result)
                if dict_result.get('fairness_bias_mentioned') == 'yes':
                    positives.append(dict_result)
                break  # Break the loop if the request was successful
            except Exception as e:
                if attempt < retry_attempts - 1:
                    print(f"Attempt {attempt + 1}: Waiting for 2.5 seconds before retrying...")
                    time.sleep(5)  # Wait for 2.5 seconds before the next attempt
                else:
                    print(f"Failed to process item {item['name']} after {retry_attempts} attempts due to an error: {str(e)}")
                    # Optionally, you could append partial results or error information to results list here

    with open('../Data/processed_comments_2.json', 'w') as outfile:
        json.dump(results, outfile, indent=4)
    
    return results, positives

if __name__ == "__main__":
    results_2, positives_2 = process_comments(file_path_2, api_key)

{
    "category": "Natural Language Processing",
    "fairness_bias_mentioned": "no",
    "how_fairness": "n/a",
    "data_privacy": "no",
    "how_data_privacy": "n/a",
    "toy": "no",
    "red_team": "no",
    "transparency_mentioned": "no",
    "how_transparency": "n/a"
}
{
    "category": "Data Science",
    "fairness_bias_mentioned": "no",
    "how_fairness": "n/a",
    "data_privacy": "no",
    "how_data_privacy": "n/a",
    "toy": "no",
    "red_team": "no",
    "transparency_mentioned": "no",
    "how_transparency": "n/a"
}
{
    "category": "Healthcare",
    "fairness_bias_mentioned": "no",
    "how_fairness": "n/a",
    "data_privacy": "yes",
    "how_data_privacy": "The data involves de-identified medical records, which potentially mitigates privacy concerns. However, de-identification alone may not fully prevent re-identification risks.",
    "toy": "no",
    "red_team": "no",
    "transparency_mentioned": "no",
    "how_transparency": "n/a: The competition emphasizes a ra

In [43]:
len(results_2), len(positives_2)

(300, 8)

In [45]:
results_2[:3]

[{'category': 'Natural Language Processing',
  'fairness_bias_mentioned': 'no',
  'how_fairness': 'n/a',
  'data_privacy': 'no',
  'how_data_privacy': 'n/a',
  'toy': 'no',
  'red_team': 'no',
  'transparency_mentioned': 'no',
  'how_transparency': 'n/a',
  'name': 'CPROD1: Consumer PRODucts contest #1',
  'url': 'https://www.kaggle.com/competitions/cprod1'},
 {'category': 'Data Science',
  'fairness_bias_mentioned': 'no',
  'how_fairness': 'n/a',
  'data_privacy': 'no',
  'how_data_privacy': 'n/a',
  'toy': 'no',
  'red_team': 'no',
  'transparency_mentioned': 'no',
  'how_transparency': 'n/a',
  'name': 'EMC Israel Data Science Challenge',
  'url': 'https://www.kaggle.com/competitions/emc-data-science'},
 {'category': 'Healthcare',
  'fairness_bias_mentioned': 'no',
  'how_fairness': 'n/a',
  'data_privacy': 'yes',
  'how_data_privacy': 'The data involves de-identified medical records, which potentially mitigates privacy concerns. However, de-identification alone may not fully preven

In [47]:
reds = [i for i in results_2 if i['red_team'] == 'yes']
len(reds)

4

In [48]:
reds

[{'category': 'Machine Learning and AI Security',
  'fairness_bias_mentioned': 'no',
  'how_fairness': 'n/a',
  'data_privacy': 'no',
  'how_data_privacy': 'n/a',
  'toy': 'no',
  'red_team': 'yes',
  'transparency_mentioned': 'no',
  'how_transparency': 'n/a',
  'name': 'NIPS 2017: Non-targeted Adversarial Attack',
  'url': 'https://www.kaggle.com/competitions/nips-2017-non-targeted-adversarial-attack'},
 {'category': 'Machine Learning Security',
  'fairness_bias_mentioned': 'no',
  'how_fairness': 'n/a',
  'data_privacy': 'no',
  'how_data_privacy': 'n/a',
  'toy': 'no',
  'red_team': 'yes',
  'transparency_mentioned': 'no',
  'how_transparency': 'n/a',
  'name': 'NIPS 2017: Targeted Adversarial Attack',
  'url': 'https://www.kaggle.com/competitions/nips-2017-targeted-adversarial-attack'},
 {'category': 'Machine Learning/Security',
  'fairness_bias_mentioned': 'no',
  'how_fairness': 'n/a',
  'data_privacy': 'no',
  'how_data_privacy': 'n/a',
  'toy': 'no',
  'red_team': 'yes',
  'tr

In [63]:
with open('../Data/kaggle_results/red_teaming.json', 'w') as outfile:
        json.dump(reds, outfile, indent=4)

In [64]:
fairness_bias = positives_2
with open('../Data/kaggle_results/fairness_bias.json', 'w') as outfile:
        json.dump(all_positives, outfile, indent=4)

In [58]:
data_privacy = [i for i in results_2 if i['data_privacy'] == 'yes']
len(data_privacy)

68

In [65]:
with open('../Data/kaggle_results/data_privacy.json', 'w') as outfile:
    json.dump(data_privacy, outfile, indent=4)

In [60]:
transparency = [i for i in results_2 if i['transparency_mentioned'] == 'yes']
len(transparency)

4

In [66]:
with open('../Data/kaggle_results/transparency.json', 'w') as outfile:
    json.dump(transparency, outfile, indent=4)

### AI Crowd

In [57]:
file_path = '../Data/aicrowd_raw.json'
with open(file_path, 'r') as file:
        aicrowd_comp_data = json.load(file)

# aicrowd_comp_data[0]

In [70]:
def query_chatgpt_2(overview, rules, api_key):
    client = openai.OpenAI(api_key=api_key)
    
    prompt = f"""
        Analyze the information of the following data competition with the overview of the competition being: '{overview}', , and the competition rule description is: '{rules}'.

        Provide a structured output in JSON style with the following fields:

            - "category": the field or industry this competition belongs to.
            - "fairness_bias_mentioned": yes or no, depending on whether fairness and bias of the model and dataset are explicitly mentioned in the descriptions or overview.
            - "how_fairness": a summary (within 50 words) describing how fairness and bias are mentioned, or "n/a" if not mentioned.
            - "data_privacy": yes or no, depending on whether you think the data used in the competition has data privacy issues (e.g., sensitive personal information, unauthorized data use, data breaches).
            - "how_data_privacy": summarize (within 50 words) what potential data privacy issues exist, or "n/a" if there are none. If privacy issues are present, explain how they are addressed and whether there are flaws in the approach.
            - "toy": yes or no, depending on whether this is a toy competition (meant for fun or practice only, no real-world impact).
            - "red_team": yes or no, indicating if the competition is a red teaming competition.
            - "transparency_mentioned": yes or no, depending on whether model transparency and explainability are explicitly mentioned as evaluation criteria.
            - "how_transparency": summarize (within 50 words) how transparency or explainability is discussed, or "n/a" if not mentioned. If the competition only focuses on a single best performance metric without considering model transparency, tell me what the metric is after "n/a".

        Please use double quotes for all keys and string values! Do not include any other text besides the structured output! DO NOT include the string 'json' anywhere in the answer! I DO NOT WANT TO SEE 'json' anywhere in the answer! Again, 'json' is a forbidden word in the answer!!!
        """
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an assistant trained to analyze comments and provide structured outputs."},
            {"role": "user", "content": prompt}
        ]
    )

    generated_text = response.choices[0].message.content.strip()
    # Print raw text to debug
    # print("------ RAW OUTPUT FROM GPT ------")
    # print(generated_text)
    # print("---------------------------------")
    
    return json.loads(generated_text)

In [71]:
def process_comments_test(file_path, api_key, num_samples=3):
    with open(file_path, 'r') as file:
        comments_data = json.load(file)

    # Take only the first `num_samples` competitions
    sample_data = comments_data[:num_samples]

    results = []

    for item in sample_data:
        retry_attempts = 150
        for attempt in range(retry_attempts):
            try:
                dict_result = query_chatgpt_2(item['overview'], item['rules'], api_key)
                dict_result['url'] = item['url']
                results.append(dict_result)
                break  # Break retry loop if successful
            except Exception as e:
                if attempt < retry_attempts - 1:
                    print(f"Attempt {attempt + 1}: Waiting for 3 seconds before retrying...")
                    time.sleep(3)
                else:
                    print(f"Failed after {retry_attempts} attempts: {str(e)}")

    # Print the results to visually check
    for idx, res in enumerate(results, 1):
        print(f"\n--- Result for Competition {idx} ---")
        print(json.dumps(res, indent=4))

    return results

# Example usage
ai_crowd_file_path = '../Data/aicrowd_raw.json'

if __name__ == "__main__":
    test_results = process_comments_test(ai_crowd_file_path, api_key)



--- Result for Competition 1 ---
{
    "category": "Artificial Intelligence and Machine Learning",
    "fairness_bias_mentioned": "no",
    "how_fairness": "n/a",
    "data_privacy": "yes",
    "how_data_privacy": "Participants must adhere to ethical standards and licensing terms for external datasets, ensuring data use transparency. Privacy is addressed by documentation requirements, but potential issues exist if these guidelines are not rigorously enforced.",
    "toy": "no",
    "red_team": "no",
    "transparency_mentioned": "yes",
    "how_transparency": "Participants must open-source their training and inference code for reproducibility. Transparency is mentioned as releasing code under an OSI-approved license and providing documentation.",
    "url": "https://www.aicrowd.com/challenges/commonsense-persona-grounded-dialogue-challenge-2025"
}

--- Result for Competition 2 ---
{
    "category": "Data Science and Machine Learning",
    "fairness_bias_mentioned": "no",
    "how_fair

In [72]:
def process_comments(file_path, api_key):
    with open(file_path, 'r') as file:
        comments_data = json.load(file)

    results = []

    # Prepare lists to collect categorized results
    fairness_bias_list = []
    red_team_list = []
    data_privacy_list = []
    transparency_list = []

    for item in comments_data:
        retry_attempts = 150
        for attempt in range(retry_attempts):
            try:
                dict_result = query_chatgpt_2(item['overview'], item['rules'], api_key)
                dict_result['url'] = item['url']
                results.append(dict_result)

                # Categorize results
                if dict_result.get('fairness_bias_mentioned', '').lower() == 'yes':
                    fairness_bias_list.append(dict_result)
                if dict_result.get('red_team', '').lower() == 'yes':
                    red_team_list.append(dict_result)
                if dict_result.get('data_privacy', '').lower() == 'yes':
                    data_privacy_list.append(dict_result)
                if dict_result.get('transparency_mentioned', '').lower() == 'yes':
                    transparency_list.append(dict_result)

                break  # Break out of retry loop if successful
            except Exception as e:
                if attempt < retry_attempts - 1:
                    print(f"Attempt {attempt + 1}: Waiting for 3 seconds before retrying...")
                    time.sleep(3)
                else:
                    print(f"Failed after {retry_attempts} attempts: {str(e)}")

    # Write categorized files
    output_dir = '../aicrowd_results'

    os.makedirs(output_dir, exist_ok=True)

    with open(os.path.join(output_dir, 'fairness_bias.json'), 'w') as fb_file:
        json.dump(fairness_bias_list, fb_file, indent=4)

    with open(os.path.join(output_dir, 'red_teaming.json'), 'w') as rt_file:
        json.dump(red_team_list, rt_file, indent=4)

    with open(os.path.join(output_dir, 'data_privacy.json'), 'w') as dp_file:
        json.dump(data_privacy_list, dp_file, indent=4)

    with open(os.path.join(output_dir, 'transparency.json'), 'w') as tr_file:
        json.dump(transparency_list, tr_file, indent=4)

    # Optionally save all processed results as a backup
    with open(os.path.join(output_dir, 'all_processed_aicrowd.json'), 'w') as all_file:
        json.dump(results, all_file, indent=4)

    return results

# Example usage
ai_crowd_file_path = '../Data/aicrowd_raw.json'

if __name__ == "__main__":
    results_ai_crowd = process_comments(ai_crowd_file_path, api_key)


In [75]:
with open('../Data/aicrowd_results/fairness_bias.json', 'r') as f:
    fairness_data = json.load(f)
with open('../Data/aicrowd_results/data_privacy.json', 'r') as f:
    privacy_data = json.load(f)
with open('../Data/aicrowd_results/red_teaming.json', 'r') as f:
    redT_data = json.load(f)
with open('../Data/aicrowd_results/transparency.json', 'r') as f:
    transparency_data = json.load(f)

print("Fairness and bias number: ", len(fairness_data))
print("Data privacy number: ", len(privacy_data))
print("Red teaming number: ", len(redT_data))
print("Transparency number: ", len(transparency_data))

Fairness and bias number:  8
Data privacy number:  42
Red teaming number:  1
Transparency number:  11


In [76]:
with open('../Data/aicrowd_results/all_processed_aicrowd.json', 'r') as f:
    all_data = json.load(f)
    
print(len(all_data))

221


## DrivenData

In [106]:
with open("../Data/drivendata_raw_scraping.json", "r") as f:
    data = json.load(f)

link_count = len(data)
print(f"Number of links: {link_count}")

urls = []
for url, text in data.items():
    urls.append(url)

with open("../Data/drivendata_urls.json", "w") as outfile:
    json.dump(urls, outfile, indent=4)

Number of links: 103


In [113]:
def filter_competitions(input_path, output_path):
    with open(input_path, "r") as f:
        data = json.load(f)

    filtered = []
    for comp in data:
        # Check if all three are "not found"
        if (
            comp["competition_overview"] == "Competition overview not found"
            and comp["dataset_overview"] == "Dataset overview not found"
            and comp["evaluation_overview"] == "Evaluation overview not found"
        ):
            continue  # skip this competition
        filtered.append(comp)

    # Save filtered results
    with open(output_path, "w") as f:
        json.dump(filtered, f, indent=4)

    print(f"Competitions left after filtering: {len(filtered)}")

if __name__ == "__main__":
    input_file = "../Data/drivendata_scraped_results.json"
    output_file = "../Data/drivendata_filtered_results.json"
    filter_competitions(input_file, output_file)


Competitions left after filtering: 65


In [104]:
with open("../Data/drivendata_filtered_results.json", 'r') as file:
    comp_data = json.load(file)

comp_overview = comp_data[0]['competition_overview']
dataset = comp_data[0]['dataset_overview']
evaluation = comp_data[0]['evaluation_overview']

# print(comp_overview, dataset, evaluation)

In [2]:
import re

def extract_json_block(text):
    # Use regex to find the first JSON-like block
    match = re.search(r'\{.*\}', text, re.DOTALL)
    if match:
        return match.group(0)
    else:
        raise ValueError("No valid JSON block found in text.")


In [3]:
def query_chatgpt_3(overview, dataset, evaluation, url, api_key):
    client = openai.OpenAI(api_key=api_key)

    prompt = f"""
    Analyze the following data competition. The competition overview is: '{overview}'. The dataset description is: '{dataset}'. The evaluation overview is: '{evaluation}'.

    Provide a structured output in JSON style with the following fields:

    - "category": the field or industry this competition belongs to.
    - "fairness_bias_mentioned": yes or no, depending on whether fairness and bias of the model and dataset are explicitly mentioned in the description.
    - "how_fairness": a summary (within 50 words) describing how fairness and bias are mentioned, or "n/a" if not mentioned.
    - "data_privacy": yes or no, depending on whether you think the data used in the competition has data privacy issues (e.g., sensitive personal information, unauthorized data use, data breaches).
    - "how_data_privacy": summarize (within 50 words) what potential data privacy issues exist, or "n/a" if there are none. If privacy issues are present, explain how they are addressed and whether there are flaws in the approach.
    - "toy": yes or no, depending on whether this is a toy competition (meant for fun or practice only, no real-world impact).
    - "red_team": yes or no, indicating if the competition is a red teaming competition.
    - "transparency_mentioned": yes or no, depending on whether model transparency and explainability are explicitly mentioned as evaluation criteria.
    - "how_transparency": summarize (within 50 words) how transparency or explainability is discussed, or "n/a" if not mentioned. If the competition only focuses on a single best performance metric without considering model transparency, tell me what the metric is after "n/a".

    Please use double quotes for all keys and string values! Do not include any other text besides the structured output! DO NOT include the string 'json' anywhere in the answer! I DO NOT WANT TO SEE 'json' anywhere in the answer! Again, 'json' is a forbidden word in the answer!!!
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an assistant trained to analyze competitions and provide structured outputs."},
            {"role": "user", "content": prompt}
        ]
    )

    generated_text = response.choices[0].message.content.strip()

    print("------ RAW OUTPUT FROM GPT ------")
    print(generated_text)
    print("---------------------------------")

    cleaned_json_text = extract_json_block(generated_text)

    result = json.loads(cleaned_json_text)
    result["url"] = url
    return result


In [4]:
def process_comments_drivendata(file_path, api_key):
    with open(file_path, 'r') as file:
        data = json.load(file)

    all_results = []
    fairness_bias_list = []
    red_team_list = []
    data_privacy_list = []
    transparency_list = []

    for comp in data:
        overview = comp.get("comp_overview", "")
        dataset = comp.get("data_overview", "")
        evaluation = comp.get("eval_overview", "")
        url = comp.get("url", "")

        retry_attempts = 10
        for attempt in range(retry_attempts):
            try:
                result = query_chatgpt_3(overview, dataset, evaluation, url, api_key)
                all_results.append(result)

                if result.get('fairness_bias_mentioned', '').lower() == 'yes':
                    fairness_bias_list.append(result)
                if result.get('red_team', '').lower() == 'yes':
                    red_team_list.append(result)
                if result.get('data_privacy', '').lower() == 'yes':
                    data_privacy_list.append(result)
                if result.get('transparency_mentioned', '').lower() == 'yes':
                    transparency_list.append(result)

                break  # Success
            except Exception as e:
                if attempt < retry_attempts - 1:
                    print(f"Attempt {attempt + 1}: Waiting for 3 seconds before retrying...")
                    time.sleep(3)
                else:
                    print(f"Failed after {retry_attempts} attempts: {str(e)}")

    # Write results
    output_dir = '../Data/drivendata_results'
    os.makedirs(output_dir, exist_ok=True)

    with open(os.path.join(output_dir, 'fairness_bias.json'), 'w') as fb_file:
        json.dump(fairness_bias_list, fb_file, indent=4)

    with open(os.path.join(output_dir, 'red_teaming.json'), 'w') as rt_file:
        json.dump(red_team_list, rt_file, indent=4)

    with open(os.path.join(output_dir, 'data_privacy.json'), 'w') as dp_file:
        json.dump(data_privacy_list, dp_file, indent=4)

    with open(os.path.join(output_dir, 'transparency.json'), 'w') as tr_file:
        json.dump(transparency_list, tr_file, indent=4)

    with open(os.path.join(output_dir, 'all_processed_drivendata.json'), 'w') as all_file:
        json.dump(all_results, all_file, indent=4)

    print(f"Finished processing. Total competitions analyzed: {len(all_results)}")
    return all_results


In [12]:
if __name__ == "__main__":
    drivendata_file_path = '../Data/drivendata_results/all_comps.json'
    results_drivendata = process_comments_drivendata(drivendata_file_path, api_key)

    # Print first few results to check
    n_preview = 3
    for i, res in enumerate(results_drivendata[:3], start=1):
        print(f"\n----- Preview Result {i} -----")
        print(json.dumps(res, indent=4))

------ RAW OUTPUT FROM GPT ------
{
    "category": "Artificial Intelligence and Computer Vision",
    "fairness_bias_mentioned": "no",
    "how_fairness": "n/a",
    "data_privacy": "yes",
    "how_data_privacy": "The dataset is derived from YFCC100M, which may include publicly available data, but there is no mention of handling sensitive information or permissions, indicating potential privacy issues.",
    "toy": "no",
    "red_team": "no",
    "transparency_mentioned": "no",
    "how_transparency": "n/a. The competition focuses on micro-average precision as the key performance metric."
}
---------------------------------
------ RAW OUTPUT FROM GPT ------
{
    "category": "Privacy-enhancing technologies and federated learning",
    "fairness_bias_mentioned": "no",
    "how_fairness": "n/a",
    "data_privacy": "yes",
    "how_data_privacy": "The competition involves analyzing sensitive data, such as financial and pandemic-related data. Privacy issues are addressed by using federate

In [13]:
with open('../Data/drivendata_results/fairness_bias.json', 'r') as f:
    fairness_data = json.load(f)
with open('../Data/drivendata_results/data_privacy.json', 'r') as f:
    privacy_data = json.load(f)
with open('../Data/drivendata_results/red_teaming.json', 'r') as f:
    redT_data = json.load(f)
with open('../Data/drivendata_results/transparency.json', 'r') as f:
    transparency_data = json.load(f)

print("Fairness and bias number: ", len(fairness_data))
print("Data privacy number: ", len(privacy_data))
print("Red teaming number: ", len(redT_data))
print("Transparency number: ", len(transparency_data))

Fairness and bias number:  5
Data privacy number:  21
Red teaming number:  2
Transparency number:  5


In [14]:
with open('../Data/kaggle_results/all_processed_kaggle.json', 'r') as f:
    kaggle_all = json.load(f)
with open('../Data/aicrowd_results/all_processed_aicrowd.json', 'r') as f:
    aicrowd_all = json.load(f)
with open('../Data/drivendata_results/all_processed_drivendata.json', 'r') as f:
    drivendata_all = json.load(f)

print(f"All Kaggle competition analyzed: {len(kaggle_all)}.")
print(f"All AI Crowd competition analyzed: {len(aicrowd_all)}.")
print(f"All Driven Data competition analyzed: {len(drivendata_all)}.")
print(f"Total competitions analyzed: {len(kaggle_all) + len(aicrowd_all) + len(drivendata_all)}.")

All Kaggle competition analyzed: 300.
All AI Crowd competition analyzed: 221.
All Driven Data competition analyzed: 66.
Total competitions analyzed: 587.
